In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from nbdev import *
%nbdev_default_export physics
%nbdev_default_class_level 3

Cells will be exported to geomechy.physics,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from geomechy.base import BaseElement
from geomechy.core import NodeSet

# Physics
> Main problems

## Small Strain

In [ ]:
%nbdev_export
class SmallStrainContinuum(BaseElement):
    
    dofTypes = ['u', 'v']
  
    def __init__(self, nodes, props):
        BaseElement.__init__(self, nodes, props)

    def __type__(self):
        return name

In [ ]:
from geomechy.base import Properties, ItemDict

props = Properties()
props = Properties({'type':'Truss',  'material':{'type':'Elastic','E':500 , 'nu':1./3}})

nodes = NodeSet()

nodes.add( 1, [  0.,0.  ] ) #Attachement point of spring (can be positioned anywhere along the y-axis)
nodes.add( 2, [-10.,0.  ] ) #Left support
nodes.add( 3, [ 10.,0.  ] ) #Right support
nodes.add( 4, [ 0. ,0.5 ] ) #Loading point
nodes

soil = SmallStrainContinuum(nodes, props)
soil

ValueError: too many values to unpack (expected 2)

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted index.ipynb.
